In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils
from pathlib import Path


from itertools import product


import pandas as pd

import os

import sys
sys.path.append('../../../..')
from utils import load_results, merge_guarantees

import pickle

#from group_amplification.privacy_analysis.utils import get_privacy_spent
from warnings import warn
from numpy.typing import NDArray

In [2]:

def get_privacy_spent(
    *, orders: list[float] | float, rdp: list[float] | float, delta: float
) -> tuple[float, float]:
    """
    This code is taken from Opacus, which is available under Apache 2.0 license.
    For the camera ready version, we will provide our own implementation
    that does not rely on external code.

    https://github.com/pytorch/opacus/blob/main/opacus/accountants/analysis/rdp.py
    
    Computes epsilon given a list of Renyi Differential Privacy (RDP) values at
    multiple RDP orders and target ``delta``.
    The computation of epslion, i.e. conversion from RDP to (eps, delta)-DP,
    is based on the theorem presented in the following work:
    Borja Balle et al. "Hypothesis testing interpretations and Renyi differential privacy."
    International Conference on Artificial Intelligence and Statistics. PMLR, 2020.
    Particullary, Theorem 21 in the arXiv version https://arxiv.org/abs/1905.09982.
    Args:
        orders: An array (or a scalar) of orders (alphas).
        rdp: A list (or a scalar) of RDP guarantees.
        delta: The target delta.
    Returns:
        Pair of epsilon and optimal order alpha.
    Raises:
        ValueError
            If the lengths of ``orders`` and ``rdp`` are not equal.
    """
    orders_vec = np.atleast_1d(orders)
    rdp_vec = np.atleast_1d(rdp)

    if len(orders_vec) != len(rdp_vec):
        raise ValueError(
            f"Input lists must have the same length.\n"
            f"\torders_vec = {orders_vec}\n"
            f"\trdp_vec = {rdp_vec}\n"
        )

    eps = (
        rdp_vec
        - (np.log(delta) + np.log(orders_vec)) / (orders_vec - 1)
        + np.log((orders_vec - 1) / orders_vec)
    )

    # special case when there is no privacy
    if np.isnan(eps).all():
        return np.inf, np.nan

    idx_opt = np.nanargmin(eps)  # Ignore NaNs
    if idx_opt == 0 or idx_opt == len(eps) - 1:
        extreme = "smallest" if idx_opt == 0 else "largest"
        warn(
            f"Optimal order is the {extreme} alpha. Please consider expanding the range of alphas to get a tighter privacy bound."
        )
    return eps[idx_opt], orders_vec[idx_opt]

In [3]:
collection = 'group_amplification_neurips24_rdp'


jk_config = {
    'username': 'YOURUSERNAME',
    'password': 'YOURPASSWORD',
    'host': 'YOURDATABASEHOST',
    'port': 27017,
    'db_name': 'YOURDATABASENAME'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)

In [4]:
def get_experiments(col, restrictions={}):
    
    restrictions['status'] = 'COMPLETED'

    if col.count_documents(restrictions) == 0:
        raise ValueError('No matches!')

    exps = col.find(restrictions, {'config':1, 'result': 1, '_id': 1})
    
    return exps

In [5]:
def get_dp_guarantees(save_file):
    with open(save_file, 'rb') as f:
        results = pickle.load(f)

    return {
        'alphas': np.array(results['alphas']),
        'epsilons': np.array(results['epsilons'])
    }

In [6]:
def generate_exp_result_dict(exp):

    result_dict = {}

    
    result_dict['space'] = exp['config']['alphas']['space']
    result_dict['true_response_prob'] = exp['config']['base_mechanism']['params']['true_response_prob']
    result_dict['subsampling_rate'] = exp['config']['amplification']['params']['subsampling_rate']

    result_dict['group_size'] = exp['config']['amplification']['params']['group_size']
    result_dict['insertions'] = exp['config']['amplification']['params']['insertions']

    result_dict['tight'] = bool(exp['config']['amplification']['tight'])
    result_dict['eval_method'] = exp['config']['amplification']['params']['eval_method']

    save_file = exp['result']['save_file']

    result_dict['raw_results_file'] = save_file

    dp_dict = get_dp_guarantees(result_dict['raw_results_file'])

    result_dict.update(dp_dict)

    return result_dict

In [ ]:
experiments = get_experiments(col, {'config.amplification.subsampling_scheme': 'poisson',
                                    'config.base_mechanism.name': 'randomizedresponse',
                                    'config.alphas.space': {'$in': ['log', 'log_continuous', 'log_linear']}
                                    })
results = load_results(
            generate_exp_result_dict,
            experiments,
            results_file='./raw_data_gaussian',
            overwrite=False
            )

results = results.loc[results['eval_method'].isin(['recursive', 'quadrature'])]
results = results.loc[results['group_size'].isin([1, 2, 4, 8])]
results = results.loc[results['true_response_prob'].isin([0.6])]
results = results.loc[results['subsampling_rate'].isin([0.1, 0.001])]


In [ ]:
results

In [9]:
method_label_map = {
        'recursive': 'Post-hoc',
        'expansion': 'Specific',
    }

In [10]:
def prepare_plot_dict(data):

    plot_dict = {}

    for i, (index, row) in enumerate(data.iterrows()):
        alphas, epsilons, eval_method, group_size = row.loc[['alphas', 'epsilons', 'eval_method', 'group_size']]

        assert eval_method in ['recursive', 'expansion', 'quadrature']

        if eval_method == 'recursive':
            # Renyi-divergence is non-decreasing --> Make values smaller to favor baseline
            epsilons = np.minimum.accumulate(epsilons[::-1])[::-1]
        
        if eval_method == 'quadrature':
            eval_method = 'expansion'

        if eval_method not in plot_dict:
            plot_dict[eval_method] = {
                group_size: (alphas, epsilons),
                'label': method_label_map[eval_method]
            }
        
        elif group_size not in plot_dict[eval_method]:

            plot_dict[eval_method][group_size] = alphas, epsilons

        else:
            old_alphas, old_epsilons = plot_dict[eval_method][group_size]
            merged_alphas, merged_epsilons = merge_guarantees(old_alphas, alphas,
                                                              old_epsilons, epsilons,
                                                              max)
            
            if eval_method == 'recursive':
                # Renyi-divergence is non-decreasing --> Make values smaller to favor baseline
                merged_epsilons = np.minimum.accumulate(merged_epsilons[::-1])[::-1]

            plot_dict[eval_method][group_size] = merged_alphas, merged_epsilons

    return plot_dict

In [11]:
def convert_plot_dict(plot_dict, delta: float, iterations: NDArray):
    res = plot_dict.copy()

    for eval_method, eval_dict in plot_dict.items():
        for k in eval_dict:
            if isinstance(k, str):
                continue
            else:
                group_size = k
            alphas, epsilons = eval_dict[group_size]
            delta_epsilons = np.array([get_privacy_spent(orders=alphas, rdp=i * epsilons, delta=delta)[0] for i in iterations])
            res[eval_method][group_size] = (iterations, delta_epsilons)

    return res

In [12]:
def plot_plot_dict(plot_dict, draw_legend_group_size=False, draw_legend_method=False, width=0.49, xlim=[2, 10000]):
    sns.set_theme()

    fig, ax = plt.subplots()

    pal = sns.color_palette('colorblind', 4)[::-1]

    smallest_alpha = None
    largest_alpha = None

    for i, (eval_method, eval_method_dict) in list(enumerate(plot_dict.items())):
        group_sizes = np.sort([k for k in eval_method_dict if not isinstance(k, str)])

        for j, group_size in enumerate(group_sizes[::-1]):

            alphas, epsilons = eval_method_dict[group_size]
            smallest_alpha = alphas.min() if smallest_alpha is None else min(smallest_alpha, alphas.min())

            prob_label = group_size if eval_method == 'expansion' else None

            linestyle = 'solid' if eval_method in ['quadrature', 'expansion'] else 'dashed'

            ax.plot(alphas, epsilons, label=prob_label, c=pal[int(np.log2(group_size)) - 1], linestyle=linestyle)

    ax.set_xscale('log')
    ax.set_yscale('log')

    ax.tick_params('both', which='major', length=2.5, width=0.75)
    ax.tick_params('both', which='minor', length=1.5, width=0.75, left=True)

    ax.set_xlabel('Iteration $t$', fontsize=9)
    ax.set_ylabel('ADP $\\varepsilon$', fontsize=9)

    if draw_legend_group_size:
        legend_group_size = ax.legend(loc='lower right', title='Group size', title_fontsize=9)

    if draw_legend_method:
        handles_ls = []
        handles_ls.append(ax.plot([], [], c='black', ls='dashed')[0])
        handles_ls.append(ax.plot([], [], c='black', ls='solid')[0])

        legend_method = ax.legend(handles_ls, list(method_label_map.values()), loc=('upper left' if True else 'lower right'))

        if draw_legend_group_size:
            ax.add_artist(legend_group_size)

In [ ]:
save_dir = '/ceph/hdd/staff/schuchaj/group_amplification_plots/neurips24/rdp/poisson/specific_vs_posthoc_accounting/randomized_response/half_page/both_legends'

for x in results.groupby(['subsampling_rate', 'true_response_prob']):

    subsampling_rate, true_response_prob = x[0]
    plot_dict = prepare_plot_dict(x[1])

    print(subsampling_rate, true_response_prob)
    plot_dict = convert_plot_dict(plot_dict, 1e-8, np.logspace(0, 5, 121))

    draw_legend_method = True
    draw_legend_group_size = True

    plot_plot_dict(plot_dict, draw_legend_method=draw_legend_method, draw_legend_group_size=draw_legend_group_size, width=0.49, xlim=None)

    plt.savefig(f'{save_dir}/{subsampling_rate}_{true_response_prob}.png', dpi=256)

    #plt.close()    

In [ ]:
save_dir = '/ceph/hdd/staff/schuchaj/group_amplification_plots/neurips24/rdp/poisson/specific_vs_posthoc_accounting/randomized_response/half_page/no_legend'

for x in results.groupby(['subsampling_rate', 'true_response_prob']):

    subsampling_rate, true_response_prob = x[0]
    plot_dict = prepare_plot_dict(x[1])

    print(subsampling_rate, true_response_prob)
    plot_dict = convert_plot_dict(plot_dict, 1e-8, np.logspace(0, 5, 121))

    draw_legend_method = False
    draw_legend_group_size = False

    plot_plot_dict(plot_dict, draw_legend_method=draw_legend_method, draw_legend_group_size=draw_legend_group_size, width=0.49, xlim=None)

    plt.savefig(f'{save_dir}/{subsampling_rate}_{true_response_prob}.png', dpi=256)

    #plt.close()    